In [1]:
using Enzyme
using LinearAlgebra

In [2]:
function detCm1(E2)
    det_E2 = det(E2)
    A1 = E2[1,1]*E2[2,2] + E2[1,1]*E2[3,3] + E2[2,2]*E2[3,3]
    A2 = E2[1,2]*E2[2,1] + E2[1,3]*E2[3,1] + E2[2,3]*E2[3,2]
    # det(C - 1)
    det_E2 + tr(E2) + A1 - A2
end

detCm1 (generic function with 1 method)

In [3]:
function StrainEnergy!(strain_energy, E2work, mu)
    
    E2 = [ 
              E2work[1] E2work[6] E2work[5];
              E2work[6] E2work[2] E2work[4];
              E2work[5] E2work[4] E2work[3]
         ]
    
    # log(J)
    det_Cm1 = detCm1(E2)
    logJ = log(det_Cm1 + 1) / 2.

    # trace(E)
    traceE = (E2work[1] + E2work[2] + E2work[3]) / 2.

    strain_energy[] = mu * (-logJ + traceE)
    
    nothing

end

StrainEnergy! (generic function with 1 method)

In [4]:
E2work = [
    0.5895232828911128,
    0.2362491738162759,
    0.9793730522395296,
    0.2190993957421843,
    0.0126503210747925,
    0.6570956167695403
];
mu = 1;

In [25]:
#strain_energy = StrainEnergy!(E2work, mu)
se = Ref(0.)
StrainEnergy!(se, E2work, mu);
se

Base.RefValue{Float64}(0.35963060590632745)

In [46]:
Stress = zeros(6)
dse = Ref(1.0)
# Skip this for now since it's doesn't handle allocations that happen above
#Enzyme.autodiff(StrainEnergy!, Const, Duplicated(se, dse), Duplicated(E2work, Stress), Const(mu))

Base.RefValue{Float64}(1.0)

## Enzyme gradient with StaticArrays

In [23]:
using StaticArrays

E = SMatrix{3,3}(.5*reshape(E2work[[1,6,5,6,2,4,5,4,3]], 3, 3))

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
 0.294762    0.328548  0.00632516
 0.328548    0.118125  0.10955
 0.00632516  0.10955   0.489687

In [26]:
psi(E) = mu * (-log(det(I+2E))/2 + tr(E))
psi(E)

0.35963060590632734

In [44]:
S, = autodiff(psi, Active, Active(E))
S

3×3 SMatrix{3, 3, Float64, 9} with indices SOneTo(3)×SOneTo(3):
  0.190092    0.438163  -0.0433247
  0.438163   -0.062133   0.114769
 -0.0433247   0.114769   0.482363

## Zygote gradient

In [45]:
import Zygote
S, = Zygote.gradient(psi, E)
S

3×3 SizedMatrix{3, 3, Float64, 2, Matrix{Float64}} with indices SOneTo(3)×SOneTo(3):
  0.190092    0.438163  -0.0433247
  0.438163   -0.062133   0.114769
 -0.0433247   0.114769   0.482363